In [ ]:
from scipy.stats import boxcox
from scipy.special import inv_boxcox

plt.figure()
plt.title('registered vehicles')
train['registered_vehicles'].hist()

y,train_fitted_lambda= boxcox(train['registered_vehicles'],lmbda=None)

plt.figure()
plt.title('box cox transformation of registered vehicles')
sns.distplot(y)
print(train_fitted_lambda)

plt.figure()
plt.title('sqrt transformation of registered vehicles')
np.sqrt(train['registered_vehicles']).hist()

box-cox변환

In [ ]:
model = XGBRegressor(tree_method='gpu_hist')

param_lst = {
    'learning_rate' : [0.01, 0.1, 0.15, 0.3, 0.5],
    'n_estimators' : [100,200],
    'max_depth' : [3, 6, 9],
    'min_child_weight' : [1, 5, 10, 20],
    'reg_alpha' : [0.001, 0.01, 0.1],
    'reg_lambda' : [0.001, 0.01, 0.1]
}

xgb_reg = RandomizedSearchCV(estimator = model, param_distributions = param_lst,
                              n_iter = 20, scoring = 'neg_mean_absolute_error',
                              cv = 5)
       
xgb_search = xgb_reg.fit(encoded_train_scaled, Y)

# XGB with tune hyperparameters
best_param = xgb_search.best_params_
model = XGBRegressor(**best_param)
print(best_param)

In [ ]:
kwargs={'reg_lambda':0.1, 'reg_alpha':0.01, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.15}

model = XGBRegressor(tree_method='gpu_hist'
                     )
model.set_params(**kwargs)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model,encoded_train_scaled, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )
model.fit(encoded_train_scaled, Y, eval_metric=['mae'])
y_pred = model.predict(encoded_test_scaled)

# reverse transformation
y_pred =inv_boxcox(y_pred, fitted_lambda)

# summarize prediction
predictions = [int(value) for value in y_pred]